## Explore, clean search/download data
**Notice both search/download data are lack from 2017-03-04 to 2017-03-29**
- extract useful variables
- clean outliers
- check whether the samped_uid is involved in search/download file?
- join cleaned file with sampled_play
- ...

### Import pyspark libs and initialze SparkSession

In [1]:
#sparksession is used to create DF, register DF as tables, excute SQL over tables, cache tables
from pyspark.sql import SparkSession

from pyspark.sql import Row
from pyspark.sql import functions

from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql.types import *

from operator import add
from operator import ge
import sys

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
#Next, create pyspark.sql.SparkSession, which is the main entry point for DataFrame and SQL functionality
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Search.log
### Load in search_file
all_search.log.fn is 953MB, can we use Pandas?
- there are search_log with file name as 1_search, 2_search,...
- Notice in search.log, there is one variable record the search time, so maybe there is no need to append file_name...but we can still keep as a reference of the accuracy of our data
- It seems we don't need url 

In [5]:
rdd = sc.textFile("D:/MusicFile/all_search_1.log.fn")
#this will read .log as rdd

In [6]:
rdd.count()

9794980

In [17]:
rdd.take(10)

['154436633 \tip \t2017-03-01 00:00:24 \t%e9%83%ad%e5%be%b7%e7%ba%b2 \t 1_1_search.log',
 '154407262 \tar \t2017-03-01 00:00:53 \t%E6%AF%AF%E5%AD%90%E8%88%9E \t 1_1_search.log',
 '154407854 \tip \t2017-03-01 00:00:54 \t%e7%96%a4%2d%20%28%e7%94%b5%e8%a7%86%e5%89%a7%e3%80%8a%e9%a3%9e%e5%88%80%e5%8f%88%e8%a7%81%e9%a3%9e%e5%88%80%e3%80%8b%e4%b8%bb%e9%a2%98%e6%9b%b2%29 \t 1_1_search.log',
 '154407252 \tar \t2017-03-01 00:00:55 \t%E6%88%91%E8%A6%81%E5%88%9B%E4%B8%9A++%E5%94%90%E8%8E%89%E7%BE%A4 \t 1_1_search.log',
 '154407327 \tar \t2017-03-01 00:00:55 \t%E4%B8%AB%E5%A4%B4++%E7%8E%8B%E7%AB%A5%E8%AF%AD \t 1_1_search.log',
 '154407255 \tip \t2017-03-01 00:00:56 \t%e5%8a%a8%e5%bf%83 \t 1_1_search.log',
 '154407261 \tip \t2017-03-01 00:00:59 \t%e8%83%8e%e6%95%99 \t 1_1_search.log',
 '154407267 \tar \t2017-03-01 00:00:59 \t%E5%81%B7%E6%83%85++mc%E5%8D%8A%E9%98%B3 \t 1_1_search.log',
 '154407546 \tip \t2017-03-01 00:01:00 \t%e8%96%9b%e4%b9%8b%e8%b0%a6 \t 1_1_search.log',
 '154407254 \tar \t2017-03

In [6]:
def parseLine(line):
    fields = line.split("\t")
    if len(fields) == 5:
        try:
            uid = float(fields[0])
            device = str(fields[1])
            search_time = str(fields[2])
            url = str(fields[3])
            file_name = str(fields[4])
            return Row(uid, device, search_time, url, file_name)
        except:
            return Row(None)
    else:
        return Row(None)

schema = StructType([StructField('uid', FloatType(), False),
                     StructField('device', StringType(), True),
                     StructField('search_time', StringType(), False),
                     StructField('url', StringType(), True),
                     StructField('file_name', StringType(), True)])
len(schema)

5

In [7]:
search = rdd.map(parseLine).filter(lambda x: len(x)== len(schema))

In [8]:
search_df = spark.createDataFrame(search, schema).cache()
pd.DataFrame(search_df.take(10), columns = search_df.columns)

,uid,device,search_time,url,file_name
0,154436640.0,ip,2017-03-01 00:00:24,%e9%83%ad%e5%be%b7%e7%ba%b2,1_1_search.log
1,154407264.0,ar,2017-03-01 00:00:53,%E6%AF%AF%E5%AD%90%E8%88%9E,1_1_search.log
2,154407856.0,ip,2017-03-01 00:00:54,%e7%96%a4%2d%20%28%e7%94%b5%e8%a7%86%e5%89%a7%...,1_1_search.log
3,154407248.0,ar,2017-03-01 00:00:55,%E6%88%91%E8%A6%81%E5%88%9B%E4%B8%9A++%E5%94%9...,1_1_search.log
4,154407328.0,ar,2017-03-01 00:00:55,%E4%B8%AB%E5%A4%B4++%E7%8E%8B%E7%AB%A5%E8%AF%AD,1_1_search.log
5,154407248.0,ip,2017-03-01 00:00:56,%e5%8a%a8%e5%bf%83,1_1_search.log
6,154407264.0,ip,2017-03-01 00:00:59,%e8%83%8e%e6%95%99,1_1_search.log
7,154407264.0,ar,2017-03-01 00:00:59,%E5%81%B7%E6%83%85++mc%E5%8D%8A%E9%98%B3,1_1_search.log
8,154407552.0,ip,2017-03-01 00:01:00,%e8%96%9b%e4%b9%8b%e8%b0%a6,1_1_search.log
9,154407248.0,ar,2017-03-01 00:01:02,%E6%9D%A5%E7%94%B5%E9%93%83%E9%9F%B3,1_1_search.log


It seems we don't need url for churn prediction, drop it

In [9]:
search_df = search_df.drop('url')
pd.DataFrame(search_df.take(10), columns = search_df.columns)

,uid,device,search_time,file_name
0,154436640.0,ip,2017-03-01 00:00:24,1_1_search.log
1,154407264.0,ar,2017-03-01 00:00:53,1_1_search.log
2,154407856.0,ip,2017-03-01 00:00:54,1_1_search.log
3,154407248.0,ar,2017-03-01 00:00:55,1_1_search.log
4,154407328.0,ar,2017-03-01 00:00:55,1_1_search.log
5,154407248.0,ip,2017-03-01 00:00:56,1_1_search.log
6,154407264.0,ip,2017-03-01 00:00:59,1_1_search.log
7,154407264.0,ar,2017-03-01 00:00:59,1_1_search.log
8,154407552.0,ip,2017-03-01 00:01:00,1_1_search.log
9,154407248.0,ar,2017-03-01 00:01:02,1_1_search.log


In [10]:
#change datatype of search_time, split date and time
from pyspark.sql.types import DataType

#split_date = func.split(search_df['search_time'] " ")
search_df_1 = search_df.withColumn('record_date', search_df['search_time'].cast(DateType())) \
                       .withColumn('record_time', to_timestamp(search_df['search_time'])) \
                       .drop('search_time')
pd.DataFrame(search_df_1.take(5), columns = search_df_1.columns)

,uid,device,file_name,record_date,record_time
0,154436640.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:24
1,154407264.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:53
2,154407856.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:54
3,154407248.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:55
4,154407328.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:55


In [11]:
search_df_1.schema

StructType(List(StructField(uid,FloatType,false),StructField(device,StringType,true),StructField(file_name,StringType,true),StructField(record_date,DateType,true),StructField(record_time,TimestampType,true)))

In [22]:
#check how many unique uid in search.log?
search_uid = search_df_1.select('uid').distinct()
search_uid.count()

128386

In [72]:
#top dates with largest search time
search_df_1.groupBy('record_date').count().orderBy('count', ascending = False).show(20)

+-----------+-------+
|record_date|  count|
+-----------+-------+
| 2017-03-30|1227156|
| 2017-03-01| 685870|
| 2017-03-31| 510746|
| 2017-04-01| 351061|
| 2017-04-02| 328088|
| 2017-04-03| 289248|
| 2017-04-04| 274964|
| 2017-05-08| 241504|
| 2017-04-08| 226179|
| 2017-04-05| 220321|
| 2017-04-09| 218917|
| 2017-03-02| 213449|
| 2017-04-06| 204615|
| 2017-04-07| 202929|
| 2017-04-15| 191644|
| 2017-04-16| 188333|
| 2017-04-10| 176189|
| 2017-04-23| 169175|
| 2017-04-13| 167287|
| 2017-04-12| 163474|
+-----------+-------+
only showing top 20 rows



In [75]:
# how different between two device?
search_df_1.groupBy('uid','device').count().show()

+------------+------+-----+
|         uid|device|count|
+------------+------+-----+
|1.54416048E8|   ip |    2|
| 1.5440984E8|   ar |  189|
|1.54410512E8|   ip |    9|
|1.54413248E8|   ar |   20|
| 1.5441344E8|   ar |   33|
| 1.5441728E8|   ar |   53|
|1.54417584E8|   ar |   11|
| 7.9619968E7|   ar |    3|
|1.54420928E8|   ar |   25|
| 1.5442848E8|   ip |    7|
| 1.5443688E8|   ar |   26|
| 5.6224608E7|   ar |    8|
|1.54439184E8|   ar |    3|
|1.54443808E8|   ar |    5|
|1.54444752E8|   ip |    3|
|1.54444768E8|   ip |    1|
|1.54446736E8|   ar |   43|
|1.54447552E8|   ar |  109|
|1.54454496E8|   ip |    7|
|1.54454704E8|   ip |    3|
+------------+------+-----+
only showing top 20 rows



In [12]:
#save cleaned search.log
search_df_1.repartition(1).write.csv('D:/MusicFile/cleaned_search', header = True)

# Download.log
download.log is only 545MB, use pandas to read it.

Notice there are some Japanese letters or some encoding method other than 'utf-8', also the datatype of each fields also need to be set by us.

Most variables are similar to play.log, only lack of play_time and song_length, still, for churn analysis, we only care about the activity made by user during that period, but not what exact information about song itself. So drop those two columns.

In [51]:
col_names = ['uid', 'device', 'song_id', 'song_name', 'singer', 'paid_falg', 'file_name']
download_pd = pd.read_csv("D:/MusicFile/all_download_1.log.fn", header = None, names = col_names, encoding = 'latin-1')
download_pd.head()

,uid,device,song_id,song_name,singer,paid_flag,file_name
0,168019810.0,ar,442554.0,å°é çª,æä¿æ°&è¡åå¦,0,20170330_1_down.log
1,168019810.0,ar,6334611.0,ç¤¾ä¼æ,è§å ¨,0,20170330_1_down.log
2,168019810.0,ar,9867382.0,å°é¶,æ­¦èº,0,20170330_1_down.log
3,168019810.0,ar,6660691.0,ä¸æ¬¡å°±å¥½-(çµå½±ãå¤æ´ç¹ç¦æ¼ãææ°...,æ¨å®çº¬,0,20170330_1_down.log
4,168019810.0,ar,157606.0,ä¸è·¯ä¸æä½,å¼ å­¦å,0,20170330_1_down.log


There are multiple languages in song_name and singer, which are not encoded with utf-8, so try other encoding method..although the file size is not that big, but we'd like to use the method clean way to deal with all three log
### Cleanning download.log in pyspark

In [13]:
rdd_d = sc.textFile("D:/MusicFile/all_download_1.log.fn")
#this will read .log as rdd

In [27]:
rdd_d.count()

7746631

In [10]:
rdd_d.take(10)

['168019810 \tar \t442554 \t小酒窝 \t林俊杰&蔡卓妍 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t6334611 \t社会摇 \t萧全 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t9867382 \t台阶 \t武艺 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t6660691 \t一次就好-(电影《夏洛特烦恼》暖水曲) \t杨宗纬 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t157606 \t一路上有你 \t张学友 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t3372481 \t十年 (OT：明年今日) \t陈奕迅 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t3216525 \t你不知道的事(Live版) \t王力宏 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t6427523 \tVictory \tTwo Steps From Hell \t0 \t 20170330_1_down.log',
 '168019810 \tar \t6538686 \t野子 \t流浪者 \t0 \t 20170330_1_down.log',
 '168019810 \tar \t9327383 \tなんでもないや-[没什么大不了](Movie Ver.)-(动画电影《你的名字。》片尾曲) \tRADWIMPS \t0 \t 20170330_1_down.log']

In [14]:
def parseLine(line):
    fields = line.split("\t")
    if len(fields) == 7:
        try:
            uid = float(fields[0])
            device = str(fields[1])
            song_id = float(fields[2])
            song_name = str(fields[3])
            singer = str(fields[4])
            paid_flag = str(fields[5])
            file_name = str(fields[6])
            return Row(uid, device, song_id, song_name, singer, paid_flag, file_name)
        except:
            return Row(None)
    else:
        return Row(None)

schema = StructType([StructField('uid', FloatType(), False),
                       StructField('device', StringType(), True),
                       StructField('song_id', FloatType(), False),
                       StructField('song_name', StringType(), True),
                       StructField('singer', StringType(), True),
                       StructField('paid_flag', StringType(), True),
                       StructField('file_name', StringType(), True)])
len(schema)

7

In [15]:
download = rdd_d.map(parseLine).filter(lambda x: len(x) == 7)

In [16]:
download_df = spark.createDataFrame(download, schema).cache()
pd.DataFrame(download_df.take(10), columns = download_df.columns)

,uid,device,song_id,song_name,singer,paid_flag,file_name
0,168019808.0,ar,442554.0,小酒窝,林俊杰&蔡卓妍,0,20170330_1_down.log
1,168019808.0,ar,6334611.0,社会摇,萧全,0,20170330_1_down.log
2,168019808.0,ar,9867382.0,台阶,武艺,0,20170330_1_down.log
3,168019808.0,ar,6660691.0,一次就好-(电影《夏洛特烦恼》暖水曲),杨宗纬,0,20170330_1_down.log
4,168019808.0,ar,157606.0,一路上有你,张学友,0,20170330_1_down.log
5,168019808.0,ar,3372481.0,十年 (OT：明年今日),陈奕迅,0,20170330_1_down.log
6,168019808.0,ar,3216525.0,你不知道的事(Live版),王力宏,0,20170330_1_down.log
7,168019808.0,ar,6427523.0,Victory,Two Steps From Hell,0,20170330_1_down.log
8,168019808.0,ar,6538686.0,野子,流浪者,0,20170330_1_down.log
9,168019808.0,ar,9327383.0,なんでもないや-[没什么大不了](Movie Ver.)-(动画电影《你的名字。》片尾曲),RADWIMPS,0,20170330_1_down.log


In [20]:
download_df.select('paid_flag').distinct().count()

1

In [17]:
download_df = download_df.drop('song_name').drop('singer').drop('paid_flag')
pd.DataFrame(download_df.take(10), columns = download_df.columns)

,uid,device,song_id,file_name
0,168019808.0,ar,442554.0,20170330_1_down.log
1,168019808.0,ar,6334611.0,20170330_1_down.log
2,168019808.0,ar,9867382.0,20170330_1_down.log
3,168019808.0,ar,6660691.0,20170330_1_down.log
4,168019808.0,ar,157606.0,20170330_1_down.log
5,168019808.0,ar,3372481.0,20170330_1_down.log
6,168019808.0,ar,3216525.0,20170330_1_down.log
7,168019808.0,ar,6427523.0,20170330_1_down.log
8,168019808.0,ar,6538686.0,20170330_1_down.log
9,168019808.0,ar,9327383.0,20170330_1_down.log


In [18]:
#clean format, remove blanks
download_df2 = download_df.withColumn("device", trim(download_df.device)) \
                          .withColumn('date_str', trim(download_df.file_name.substr(1,9))) \
                          .withColumn('date_string', regexp_replace('date_str', '20170339', '20170329')) \
                          .withColumn("unix_date", unix_timestamp('date_string', 'yyyyMMdd')) \
                          .withColumn("date", from_unixtime('unix_date').cast(DateType())) \
                .drop('date_str') \
                .drop('date_string') \
                .drop('file_name') \
                .drop('unix_date')



In [20]:
pd.DataFrame(download_df2.take(10), columns = download_df2.columns)

,uid,device,song_id,date
0,168019808.0,ar,442554.0,2017-03-30
1,168019808.0,ar,6334611.0,2017-03-30
2,168019808.0,ar,9867382.0,2017-03-30
3,168019808.0,ar,6660691.0,2017-03-30
4,168019808.0,ar,157606.0,2017-03-30
5,168019808.0,ar,3372481.0,2017-03-30
6,168019808.0,ar,3216525.0,2017-03-30
7,168019808.0,ar,6427523.0,2017-03-30
8,168019808.0,ar,6538686.0,2017-03-30
9,168019808.0,ar,9327383.0,2017-03-30


In [21]:
#how many unique uid?
download_uid = download_df2.select('uid').distinct()
download_uid.count()

90281

In [70]:
#how download differ in different device?
download_df2.groupBy('device').count().show()

+------+-------+
|device|  count|
+------+-------+
|    ar|6595221|
|    ip|1132541|
+------+-------+



In [19]:
#save cleaned search.log
download_df2.repartition(1).write.csv('D:/MusicFile/cleaned_download', header = True)

**if just group by device, there maybe bias, the same ar user download 100 times**

## Compare play, search, download
1. valid_uid?
2. device  percentage?(groupby uid, device)


#### Load in valid_play_uid, sampled_uid, sampled_play 

In [4]:
valid_uid = pd.read_csv("D:/MusicFile/valid_uid/valid_uid_new.csv")
valid_uid.head()

,uid,count
0,168392096.0,5399
1,168531584.0,5398
2,168281952.0,5394
3,168373024.0,5392
4,168203056.0,5391


In [5]:
valid_uid.count()

uid      140635
count    140635
dtype: int64

valid_uid = 140635, search_uid = 128386, download_uid = 90281
check whether all search_uid and download_uid are in valid_uid


next step:
Since we have cleaned all these three log, we can now do some feature generation in pandas. As file size are all small enough for pandas.

1. merge, check valid_uid of play, download, search
2. frequency(3,7,14,30) give score?
3. model>tuning>evaluation>compare
4.

## Downsampling search and download to match the play.log 

In [5]:
#reload in sampled_play
sample_play = spark.read.csv("D:/MusicFile/sample_play_316.csv", header = True, inferSchema=True)
print(sample_play.printSchema())
#sampled_play = sampled_play.withColumnRenamed("date", "play_date")
pd.DataFrame(sample_play.take(10), columns = sample_play.columns)
#inferSchema is right, no need to set schema by ourselves

root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- song_id: double (nullable = true)
 |-- play_time: double (nullable = true)
 |-- song_length: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- churn: integer (nullable = true)

None


,uid,device,song_id,play_time,song_length,date,churn
0,49423096.0,ar,203139.0,204.0,270.0,2017-04-02,0
1,49423096.0,ar,133946.0,165.0,286.0,2017-04-02,0
2,49423096.0,ar,380389.0,40.0,233.0,2017-04-02,0
3,49423096.0,ar,23534240.0,115.0,378.0,2017-04-02,0
4,49423096.0,ar,6672344.0,69.0,249.0,2017-04-02,0
5,49423096.0,ar,6769715.0,123.0,124.0,2017-04-02,0
6,49423096.0,ar,6989315.0,80.0,84.0,2017-04-02,0
7,49423096.0,ar,7175883.0,138.0,139.0,2017-04-02,0
8,49423096.0,ar,7353175.0,6.0,123.0,2017-04-02,0
9,49423096.0,ar,921642.0,5.0,243.0,2017-04-02,0


In [6]:
sample_uid_316 = sample_play.select('uid').distinct()
sample_uid_316.count()

8823

In [7]:
#save sampled_uid as csv
sample_uid_316.repartition(1).write.csv('D:/MusicFile/sample_uid_316', header = True)

In [8]:
#reload in cleaned_search
cleaned_search = spark.read.csv("D:/MusicFile/cleaned_search_1.csv", header = True, inferSchema=True).cache()
print(cleaned_search.printSchema())
cleaned_search = cleaned_search.withColumnRenamed("record_date", "date")
pd.DataFrame(cleaned_search.take(10), columns = cleaned_search.columns)

root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- file_name: string (nullable = true)
 |-- record_date: timestamp (nullable = true)
 |-- record_time: timestamp (nullable = true)

None


,uid,device,file_name,date,record_time
0,154436640.0,ip,1_1_search.log,2017-03-01,2017-03-01 01:00:24
1,154407264.0,ar,1_1_search.log,2017-03-01,2017-03-01 01:00:53
2,154407856.0,ip,1_1_search.log,2017-03-01,2017-03-01 01:00:54
3,154407248.0,ar,1_1_search.log,2017-03-01,2017-03-01 01:00:55
4,154407328.0,ar,1_1_search.log,2017-03-01,2017-03-01 01:00:55
5,154407248.0,ip,1_1_search.log,2017-03-01,2017-03-01 01:00:56
6,154407264.0,ip,1_1_search.log,2017-03-01,2017-03-01 01:00:59
7,154407264.0,ar,1_1_search.log,2017-03-01,2017-03-01 01:00:59
8,154407552.0,ip,1_1_search.log,2017-03-01,2017-03-01 01:01:00
9,154407248.0,ar,1_1_search.log,2017-03-01,2017-03-01 01:01:02


In [9]:
#reload in cleaned_download
cleaned_download = spark.read.csv("D:/MusicFile/cleaned_download_1.csv", header = True, inferSchema=True).cache()
print(cleaned_download.printSchema())
#for the convience of following step, change date in each log into 'date'
#cleaned_download = cleaned_download.withColumnRenamed("date", "download_date")
pd.DataFrame(cleaned_download.take(10), columns = cleaned_download.columns)


root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- song_id: double (nullable = true)
 |-- date: timestamp (nullable = true)

None


,uid,device,song_id,date
0,168019808.0,ar,442554.0,2017-03-30
1,168019808.0,ar,6334611.0,2017-03-30
2,168019808.0,ar,9867382.0,2017-03-30
3,168019808.0,ar,6660691.0,2017-03-30
4,168019808.0,ar,157606.0,2017-03-30
5,168019808.0,ar,3372481.0,2017-03-30
6,168019808.0,ar,3216525.0,2017-03-30
7,168019808.0,ar,6427523.0,2017-03-30
8,168019808.0,ar,6538686.0,2017-03-30
9,168019808.0,ar,9327383.0,2017-03-30


In [10]:
sample_search = sample_uid_316.join(cleaned_search, sample_uid_316.uid == cleaned_search.uid, 'inner') \
.drop(sample_uid_316.uid)

sample_search.select('uid').distinct().count()

7714

In [11]:
#save sampled_search
sample_search.repartition(1).write.csv("D:/MusicFile/sampled_search_316", header = True)

In [12]:
sample_download = sample_uid_316.join(cleaned_download, sample_uid_316.uid == cleaned_download.uid, 'inner') \
.drop(cleaned_download.uid)
sample_download.select('uid').distinct().count()

4738

In [13]:
#save sampled_download
sample_download.repartition(1).write.csv("D:/MusicFile/sample_download_316", header = True)

In [18]:
print("search/play rate is", 7714/8823)
print("download/play rate is", 4738/8823)

print("search/play rate is", search_count/play_count)
print("download/play rate is", download_count/play_count)

search/play rate is 0.8743057916808342
download/play rate is 0.5370055536665533


#### Some Analysis
As we can see the amount of user from sampled play, to search, to download are dropping. If set play as base action, then search/play rate is 87.2%, download/play rate is 53.5%, since we are randomly sampled from the valid uid, these precentage could represent the similar rate for entire log data. If in the later, the prediction model shows search/download feature are also important for churn predictive analysis, we could think about how to increase the converse rate between each phase.